In [1]:
import numpy as np
import pandas as pd

## Read the DOE

In [2]:
df_parameters = pd.read_csv("../data/doe.csv", index_col=False)
df_parameters.keys()

Index(['doe_id', 'Material_ID', 'Blechdicke', 'Niederhalterkraft', 'Ziehspalt',
       'Stempel_ID', 'Einlegeposition', 'Ziehtiefe', 'Breite', 'UG', 'OG', 'E',
       'Material_Name', 'Rp0', 'Rp0.2', 'Rp100', 'Rp25', 'Rp50'],
      dtype='object')

In [3]:
df_parameters.head()

,doe_id,Material_ID,Blechdicke,Niederhalterkraft,Ziehspalt,Stempel_ID,Einlegeposition,Ziehtiefe,Breite,UG,OG,E,Material_Name,Rp0,Rp0.2,Rp100,Rp25,Rp50
0,1,3,1.01,410,2.4,3,-5,30,70.2,1.71,2.00,191.37245,DC04_1.00mm,138.226960,147.601859,534.002871,377.443009,449.528189
1,2,5,1.48,400,2.4,3,-4,50,70.2,1.71,2.00,71.68198,AA5083_1.50mm,235.160326,237.951493,384.123147,369.820589,383.041834
2,3,1,1.00,210,2.4,3,-1,70,70.2,1.71,2.00,164.62254,DC01_1.00mm,133.182632,147.101263,517.275855,385.773439,447.384736
3,4,2,1.19,390,2.4,3,5,30,70.2,1.71,2.00,200.03813,DC01_1.20mm,160.714935,174.535075,564.455438,424.931018,490.442223
4,5,1,1.00,360,1.6,2,3,30,71.8,1.14,1.33,164.62254,DC01_1.00mm,133.182632,147.101263,517.275855,385.773439,447.384736


## x0 cut

### with doe_id

In [3]:
# Get the parameter names
with pd.HDFStore('../data/banane_x0cut_tp_c.h5', mode="r") as store:
    
    for key in store.keys():
        
        if not key.startswith('/contours_x0cut_tp/doeid_'):
            continue

        idx = int(key[-4:])

        df_exp = store.get(key)
        features = df_exp.keys().to_list()

        break

df_x0 = pd.DataFrame(columns=['doe_id'] + features)

with pd.HDFStore('../data/banane_x0cut_tp_c.h5', mode="r") as store:
    
    for key in store.keys():
        
        if not key.startswith('/contours_x0cut_tp/doeid_'):
            continue
            
        idx = int(key[-4:])

        df_exp = store.get(key)

        df_exp.insert(0, "doe_id", idx)

        # Add to the main df
        df_x0 = pd.concat([df_x0, df_exp], ignore_index=True,)

df_x0.to_csv("../data/cut_x0_all.csv", index=False)
df_x0.head(10)

,doe_id,y,z,x,t,deviation,target_idx,y_d,z_d,x_d,t_d,c,color,deviationc,Ziehtiefe,part,tp
0,1,-94.864250,0.5,0.0,0.000000,0.543273,9,-94.855233,1.043198,0.0,0.000900,1.0,r,0.543273,30,lu,0.000000
1,1,-94.621034,0.5,0.0,0.001001,0.541849,19,-94.616495,1.041830,0.0,0.001900,1.0,r,0.541849,30,lu,0.004115
2,1,-94.377819,0.5,0.0,0.002002,0.540459,29,-94.377757,1.040459,0.0,0.002900,1.0,r,0.540459,30,lu,0.008230
3,1,-94.134603,0.5,0.0,0.003003,0.539104,39,-94.139019,1.039086,0.0,0.003900,1.0,r,0.539104,30,lu,0.012346
4,1,-93.891387,0.5,0.0,0.004004,0.537783,50,-93.876407,1.037574,0.0,0.005001,1.0,r,0.537783,30,lu,0.016461
5,1,-93.648172,0.5,0.0,0.005005,0.536301,60,-93.637669,1.036198,0.0,0.006001,1.0,r,0.536301,30,lu,0.020576
6,1,-93.404956,0.5,0.0,0.006006,0.534856,70,-93.398931,1.034822,0.0,0.007001,1.0,r,0.534856,30,lu,0.024691
7,1,-93.161740,0.5,0.0,0.007007,0.533446,80,-93.160194,1.033444,0.0,0.008001,1.0,r,0.533446,30,lu,0.028807
8,1,-92.918525,0.5,0.0,0.008008,0.532073,90,-92.921456,1.032065,0.0,0.009001,1.0,r,0.532073,30,lu,0.032922
9,1,-92.675309,0.5,0.0,0.009009,0.530738,100,-92.682718,1.030686,0.0,0.010001,1.0,r,0.530738,30,lu,0.037037


### Merge process parameters

In [ ]:
attributes_x0 = ["tp", "deviationc"]
df_x0 = pd.DataFrame(columns=df_parameters.keys().to_list() + attributes_x0)

with pd.HDFStore('../data/banane_x0cut_tp_c.h5', mode="r") as store:
    
    for key in store.keys():
        
        if not key.startswith('/contours_x0cut_tp/doeid_'):
            continue
            
        #print(key)
        idx = int(key[-4:])
        
        # Parameters
        parameters = df_parameters[df_parameters['doe_id']==idx]
        print(parameters)
        
        # Sampled values
        df_exp = store.get(key)[attributes_x0]
        
        for idx, attr in enumerate(df_parameters.keys().to_list()):
            df_exp.insert(idx, attr, parameters[attr].to_numpy()[0])
            
        # Add to the main df
        df_x0 = pd.concat([df_x0, df_exp], ignore_index=True,)
        
df_x0.to_csv("../data/cut_x0.csv", index=False)

## Web cut

### With doe_id

In [4]:
# Get the parameter names
with pd.HDFStore('../data/cut_web.h5', mode="r") as store:
    
    for key in store.keys():

        idx = int(key[-4:])

        df_exp = store.get(key)
        features = df_exp.keys().to_list()

        break

df_web = pd.DataFrame(columns=['doe_id'] + features)

with pd.HDFStore('../data/cut_web.h5', mode="r") as store:
    
    for key in store.keys():
        
            
        idx = int(key[-4:])

        df_exp = store.get(key)

        df_exp.insert(0, "doe_id", idx)

        # Add to the main df
        df_web = pd.concat([df_web, df_exp], ignore_index=True,)

df_web.to_csv("../data/cut_web_all.csv", index=False)
df_web.head(10)

,doe_id,c_phi,c_z,c_rho
0,1,1.221730,25.0,463.681714
1,1,1.225256,25.0,463.680524
2,1,1.228782,25.0,463.678661
3,1,1.232308,25.0,463.708276
4,1,1.235834,25.0,463.736364
5,1,1.239360,25.0,463.753995
6,1,1.242886,25.0,463.795277
7,1,1.246412,25.0,463.832735
8,1,1.249938,25.0,463.881185
9,1,1.253464,25.0,463.918497


### Merge process parameters

In [ ]:
attributes_web = ["c_phi", "c_rho"]
df_web = pd.DataFrame(columns=df_parameters.keys().to_list() + attributes_web)

with pd.HDFStore('../data/cut_web.h5', mode="r") as store:
    
    for key in store.keys():
        
        #print(key)
        idx = int(key[-4:])
        
        # Parameters
        parameters = df_parameters[df_parameters['doe_id']==idx]
        print(parameters)
        
        # Sampled values
        df_exp = store.get(key)[attributes_web]
        #print(df)
        for idx, attr in enumerate(df_parameters.keys().to_list()):
            df_exp.insert(idx, attr, parameters[attr].to_numpy()[0])
            
        # Add to the main df
        df_web = pd.concat([df_web, df_exp], ignore_index=True,)
        
print(df_web)
df_web.to_csv("../data/cut_web.csv", index=False)

## Flange cut

### With doe_id

In [5]:
# Get the parameter names
with pd.HDFStore('../data/cut_flange.h5', mode="r") as store:
    
    for key in store.keys():

        idx = int(key[-4:])

        df_exp = store.get(key)
        features = df_exp.keys().to_list()

        break

df_flange = pd.DataFrame(columns=['doe_id'] + features)

with pd.HDFStore('../data/cut_flange.h5', mode="r") as store:
    
    for key in store.keys():
        
            
        idx = int(key[-4:])

        df_exp = store.get(key)

        df_exp.insert(0, "doe_id", idx)

        # Add to the main df
        df_flange = pd.concat([df_flange, df_exp], ignore_index=True,)

df_flange.to_csv("../data/cut_flange_all.csv", index=False)
df_flange.head(10)

,doe_id,c_phi,c_rho,c_z
0,1,1.221730,430.0,0.579342
1,1,1.225256,430.0,0.578100
2,1,1.228782,430.0,0.576825
3,1,1.232308,430.0,0.575515
4,1,1.235834,430.0,0.574172
5,1,1.239360,430.0,0.572797
6,1,1.242886,430.0,0.571389
7,1,1.246412,430.0,0.569949
8,1,1.249938,430.0,0.568479
9,1,1.253464,430.0,0.566978


### Merge process parameters

In [ ]:
attributes_flange = ["c_phi", "c_z"]
df_flange = pd.DataFrame(columns=df_parameters.keys().to_list() + attributes_flange)

with pd.HDFStore('../data/cut_flange.h5', mode="r") as store:
    
    for key in store.keys():
        
        #print(key)
        idx = int(key[-4:])
        
        # Parameters
        parameters = df_parameters[df_parameters['doe_id']==idx]
        
        # Sampled values
        df_exp = store.get(key)[attributes_flange]

        for idx, attr in enumerate(df_parameters.keys().to_list()):
            df_exp.insert(idx, attr, parameters[attr].to_numpy()[0])
            
        # Add to the main df
        df_flange = pd.concat([df_flange, df_exp], ignore_index=True,)
        
print(df_flange)
df_flange.to_csv("../data/cut_flange.csv", index=False)